# `Close` Contract on Mainnet Using Marlowe Runtime

## Preliminaries

Record version numbers

In [1]:
marlowe-cli --version

marlowe-cli 0.0.10.0 «mainnet»


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

522050d7ac6404a2c3b02c771c6aa2620e02c627


### Setup the faucet.

Set the location of keys.

In [4]:
TREASURY=treasury

Set the faucet.

In [5]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.mainnet.address)
echo "$FAUCET_ADDR"

addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/mainnet.socket
ssh -NT \
  -L/tmp/mainnet.socket:/data/networks/mainnet/node.socket \
  -L 3717:127.0.0.1:53717 \
  -L 3718:127.0.0.1:53718 \
  -L 3719:127.0.0.1:53719 \
  -L 3721:127.0.0.1:53721 \
  -L 3723:127.0.0.1:53723 \
  54.202.238.5                                              
```

In [6]:
export CARDANO_NODE_SOCKET_PATH=/tmp/mainnet.socket
export CARDANO_TESTNET_MAGIC=64824073
MAGIC=(--mainnet)
echo "${MAGIC[@]}"

--mainnet


### The party to the contract

In [8]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [9]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx


Fund the address

In [36]:
marlowe-cli util fund-address \
  "${MAGIC[@]}" \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 10000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

361a1c7cdc64bd26f2d6054ac95643e95f36152c5b92e4933a210cec09ba8001


### Time computations

In [13]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [19]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666380791000


## The Contract

We set the parameters for the `Close` contract.

In [20]:
MINIMUM_ADA=2000000

Create the contract.

In [21]:
cat << EOI > runtime-close-1.contract
"close"
EOI
json2yaml runtime-close-1.contract

close
...


Create the initial state.

In [22]:
yaml2json << EOI > runtime-close-1.state
accounts:
- - - address: $PARTY_ADDR
    - currency_symbol: ''
      token_name: ''
  - $MINIMUM_ADA
boundValues: []
choices: []
minTime: 0
EOI
cat runtime-close-1.state

{"accounts":[[[{"address":"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":0}


## Run the Contract

### Transaction 1. Create the ontract

Build the transaction.

In [23]:
TX_1=$(
marlowe create \
  --core-file runtime-close-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign runtime-close-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

CONTRACT_ID = TX_1 = 32c3c292dc1c6194389054ca1a485cc3a6bd5c5fcb4d9120b040d91c1a56fae4#1


Sign the transaction.

In [26]:
cardano-cli transaction sign \
  --tx-body-file runtime-close-1.txbody \
  --out-file     runtime-close-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [27]:
marlowe submit runtime-close-1.tx

"{\"blockHeaderHash\":\"35668bc7cb989d1bb4783a44ad81b7b5448053eac256372afa99a4985849b013\",\"blockNo\":7913523,\"slotNo\":74815203}"


View the contract's UTxO.

In [28]:
CONTRACT_ADDR=$(marlowe-cli contract address "${MAGIC[@]}")
echo "$CONTRACT_ADDR"

addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf


In [29]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_1//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
32c3c292dc1c6194389054ca1a485cc3a6bd5c5fcb4d9120b040d91c1a56fae4     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "44a946ba1ee21f1a57a99527d8698a828217268264bacc10f9f705bcb726ad48"


Watch the contract.

In [30]:
marlowe add "$CONTRACT_ID"

32c3c292dc1c6194389054ca1a485cc3a6bd5c5fcb4d9120b040d91c1a56fae4#1


View the contract's history.

In [31]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 32c3c292dc1c6194389054ca1a485cc3a6bd5c5fcb4d9120b040d91c1a56fae4 (creation)
ContractId:      32c3c292dc1c6194389054ca1a485cc3a6bd5c5fcb4d9120b040d91c1a56fae4#1
SlotNo:          74815203
BlockNo:         7913523
BlockId:         35668bc7cb989d1bb4783a44ad81b7b5448053eac256372afa99a4985849b013
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    Close



### Transaction 2. Party deposits loan amount

The party deposits the loan amount.

In [37]:
RIGHT_NOW="$(($(date -u +%s) * SECOND))"
echo "$RIGHT_NOW"
TX_2=$(
marlowe advance \
  --contract "$CONTRACT_ID" \
  --validity-lower-bound "$((RIGHT_NOW-5*MINUTE))" \
  --validity-upper-bound "$((RIGHT_NOW+20*MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign runtime-close-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

1666383143000
ApplyFailed (ApplyInputsConstraintError (BalancingError "TxBodyErrorValidityInterval (TransactionValidityTranslationError (TimeTranslationPastHorizon \"PastHorizon {pastHorizonCallStack = [(\\\"runQuery\\\",SrcLoc {srcLocPackage = \\\"ouroboros-consensus-0.1.0.0-DMNTKru2tmj6oWAqz1LIcU\\\", srcLocModule = \\\"Ouroboros.Consensus.HardFork.History.Qry\\\", srcLocFile = \\\"src/Ouroboros/Consensus/HardFork/History/Qry.hs\\\", srcLocStartLine = 430, srcLocStartCol = 44, srcLocEndLine = 430, srcLocEndCol = 64}),(\\\"interpretQuery\\\",SrcLoc {srcLocPackage = \\\"ouroboros-consensus-0.1.0.0-DMNTKru2tmj6oWAqz1LIcU\\\", srcLocModule = \\\"Ouroboros.Consensus.HardFork.History.EpochInfo\\\", srcLocFile = \\\"src/Ouroboros/Consensus/HardFork/History/EpochInfo.hs\\\", srcLocStartLine = 45, srcLocStartCol = 50, srcLocEndLine = 45, srcLocEndCol = 68}),(\\\"interpretQuery'\\\",SrcLoc {srcLocPackage = \\\"ouroboros-consensus-0.1.0.0-DMNTKru2tmj6oWAqz1LIcU\\\", srcLocModule = \\\"Ouroboros

Sign the transaction.

In [38]:
cardano-cli transaction sign \
  --tx-body-file runtime-close-2.txbody \
  --out-file     runtime-close-2.tx \
  --signing-key-file "$PARTY_SKEY"

Command failed: transaction sign  Error: runtime-close-2.txbody: runtime-close-2.txbody: openBinaryFile: does not exist (No such file or directory)


: 1

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit runtime-close-2.tx

### Recover the funds by closing the contract using `marlowe-cli`.

In [41]:
marlowe-cli run initialize \
  --contract-file runtime-close-1.contract \
  --state-file    runtime-close-1.state    \
  --out-file      runtime-close-1.marlowe  \
  --print-stats \
  --at-address addr1z9l4w7djneh0kss4drg2php6ynflsvmal7x3w5nrc95uvhz7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqrl2z0l \
  --mainnet


Searching for reference script at address: addr1z9l4w7djneh0kss4drg2php6ynflsvmal7x3w5nrc95uvhz7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqrl2z0l

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr1z9l4w7djneh0kss4drg2php6ynflsvmal7x3w5nrc95uvhz7e4q926apsvcd6kn33cpx95k8jsmrj7v0k62rczvz8urqrl2z0l

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


In [44]:
RIGHT_NOW="$(($(date -u +%s) * SECOND))"

In [45]:
marlowe-cli run prepare \
  --marlowe-file runtime-close-1.marlowe \
  --out-file     runtime-close-2.marlowe \
  --invalid-before "$((RIGHT_NOW-1*MINUTE))" \
  --invalid-hereafter "$((RIGHT_NOW+9*MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666386189000},POSIXTime {getPOSIXTime = 1666386789999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666386189000},POSIXTime {getPOSIXTime = 1666386789999}), txInputs = []}

Datum size: 30
Payment 1
  Acccount: "\"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx\""
  Payee: Party "\"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx\""
  Ada: Lovelace {getLovelace = 2000000}


In [46]:
marlowe-cli run auto-execute \
  --mainnet \
  --tx-in-marlowe "$CONTRACT_ID" \
  --marlowe-in-file  runtime-close-1.marlowe \
  --marlowe-out-file runtime-close-2.marlowe \
  --required-signer "$PARTY_SKEY" \
  --change-address "$PARTY_ADDR" \
  --out-file /dev/null \
  --print-stats \
  --submit 600


Fee: Lovelace 425023
Size: 421 / 16384 = 2%
Execution units:
  Memory: 3074264 / 14000000 = 21%
  Steps: 810275935 / 10000000000 = 8%
0ef9b6fdbf69ebfc8218bd03cc0630fe962e7d7dd5bfd0d8701f5efe7401c675


## Cleanup

Remove the contract from history tracking.

In [47]:
marlowe rm "$CONTRACT_ID"

32c3c292dc1c6194389054ca1a485cc3a6bd5c5fcb4d9120b040d91c1a56fae4#1


In [48]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [50]:
marlowe-cli transaction simple \
  --mainnet \
  --tx-in 0ef9b6fdbf69ebfc8218bd03cc0630fe962e7d7dd5bfd0d8701f5efe7401c675#0 \
  --tx-in 0ef9b6fdbf69ebfc8218bd03cc0630fe962e7d7dd5bfd0d8701f5efe7401c675#1 \
  --tx-in 361a1c7cdc64bd26f2d6054ac95643e95f36152c5b92e4933a210cec09ba8001#1 \
  --change-address $FAUCET_ADDR \
  --required-signer $PARTY_SKEY \
  --out-file /dev/null \
  --submit 600

75fde72029192867427323afb435b2e525e531a69c54f9d3e57cdf5bb2cb50ed


In [54]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "daeca71402a7066a0e4011e5e73f24aa966cb2dfe320c9006f0b318f53aff367"


Send the funds back to the faucet.

In [56]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "debc113fbe093273567f103a009bb97be2765fe376860cd49d06378705356d30"


See that the funds have been returned to the faucet.

In [52]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
